In [1]:
import anndata, random, os, fnmatch
import scanpy as sc
import numpy as np

In [2]:
'''IMPORTING FUNCTIONS FROM GENERAL FUNCTIONS FILE'''

%run GeneralFunctions.ipynb

# Lists Used in 'DataObjectCreation' File

In [ ]:
# List of Samples and their Randomization Seeds 
sampleSeedList = [['1_VC20L', 47], ['2_AC20L', 81], ['3_VC20R', 18], ['4_AC20R', 51], ['5_VC22L', 32], ['6_AC22L', 81], 
                  ['7_VC24L', 10], ['8_AC24L', 66], ['9_VC23L', 24], ['10_VC25L', 62], ['13_AC25L', 8], ['14_AC23L', 94], 
                  ['16_VC19L', 69], ['17_VC19R', 91], ['19_AC19L', 88], ['20_AC19R', 54]]

# Functions Used in 'DataObjectCreation' File

In [ ]:
# Returns the sex of the mouse
    # sampleName = string of the sample name you want to get the information for 
def mouseSex(sampleName):
    for lst in mouseSexLst:
        mouseNum = lst[0]
        mouseSex = lst[1]
        if mouseNum in sampleName:
            return mouseSex

In [ ]:
# Returns the condition of the mouse
    # sampleName = string of the sample name you want to get the information for 
def mouseCondition(sampleName):
    for lst in sampleNameConditionLst:
        mouseCondition = lst[0]
        sample = lst[1]
        if sample == sampleName:
            return mouseCondition

In [ ]:
# This function labels each sample's halves and then concatenates the biological replaicats 
    # sample1_name = string of the name used for sample 1 
    # sample1_pathLst = list containing strings of the path of the AnnData files for sample 1 halves' 
    # sample2_name = string of the name used for sample 2
    # sample2_pathLst = list containing strings of the path of the AnnData files for sample 1 halves' 
def concatSamples(sample1_name, sample1_pathLst, sample2_name, sample2_pathLst):
        # Getting all info for sample 1
        sample1_Half1 = sc.read(sample1_pathLst[0])
        sample1_Half2 = sc.read(sample1_pathLst[1])
        
        # Getting all info for sample 2 
        sample2_Half1 = sc.read(sample2_pathLst[0])
        sample2_Half2 = sc.read(sample2_pathLst[1])
        
        # Labeling sample halves before concatonating halves
        sample1_Half1.obs['half'] = f'{sample1_name}_Half1'
        sample1_Half2.obs['half'] = f'{sample1_name}_Half2'
        sample2_Half1.obs['half'] = f'{sample2_name}_Half1'
        sample2_Half2.obs['half'] = f'{sample2_name}_Half2'
        
        # Concatenating labeled anndata objects 
        concatData = anndata.concat([sample1_Half1, sample1_Half2, sample2_Half1, sample2_Half2], join="outer")
    
        return concatData

In [ ]:
# Creates conatenated samples from a list of comparisons you want to do 
    # comparisonLst = list containing lists of the comparisons you want to make 
        # example list: [[condition of 1 and 2, sample1, sample2], [condition of 3 and 4, sample3, sample4], ...]
    # folderName = name of the folder where you want to save your concatenated samples
        # if you want to change where the files are saved completely change the 'folderPath' var inside the function
def creatingConcatSampleFiles(comparisonLst, folderName):
    # Looping through sample lists in comparisonLst 
    for sampleLst in comparisonLst:
        condition = sampleLst[0]
        sample1_name = sampleLst[1]
        sample2_name = sampleLst[2]

        # Path where the sample halves are being saved 
        folderPath = f'F:/SampleData/SampleHalves/'
        
        # Using general function 'search_files' to look for the object folder paths for the two samples
        sample1_pathLst = search_files(folderPath, sample1_name)
        sample2_pathLst = search_files(folderPath, sample2_name)

        # Using the 'concatSamples' helper function to concatenate the sample halves 
        concatData = concatSamples(sample1_name, sample1_pathLst, sample2_name, sample2_pathLst)

        # Re-Clustering
        # PCA
        sc.tl.pca(concatData, svd_solver='arpack')

        # Neighbors and UMAP Computations
        sc.pp.neighbors(concatData, n_neighbors=50, n_pcs =50, use_rep = "Scanorama")
        sc.tl.umap(concatData) 

        # Re-calculating cluster numbers 
        sc.tl.leiden(concatData, resolution = 0.3, key_added = "leiden_0.3")

        concatData.write(f'F:/SampleData/{folderName}/{condition}_{sample1_name}_{sample2_name}.h5ad')